## 使用python实现Bagging算法

### 1.Bootstrap重复抽样
我们可以通过从数据集中随机选择的行和将它们添加到一个新的列表作为新数据集的样本。我们可以重复此为固定的行数或直到新数据集的大小与原始数据集的大小的比例相匹配。

In [1]:
def bootstrapResample(dataset, ratio=1.0):
    sample = []
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

我们可以利用这个函数来估算数据集的平均值

首先我们创建一个20行的数据集，这个数据集仅包含一列，其值为0到9的随机数，并计算平均值。

然后我们可以对原始数据进行bootstrap抽样，计算平均值直到我们有一个平均值的list。计算list中的平均值应该可以对原始数据做出可靠的平均值估计。

具体的实现为，每次bootstrap的抽样抽取10%的数据作为样本，然后我们依次对原始数据集作bootstrap抽样20，40，60，80，100次，然后计算其平均值估算原始数据的平均值。

In [6]:
from random import seed
from random import random
from random import randrange

def mean(numbers):
    return sum(numbers) / float(len(numbers))

seed(42)

dataset = [[randrange(10)] for i in range(50)]
print (('True Mean: %.3f' % mean([row[0] for row in dataset])))

ratio = 0.10
for times in [20, 40, 60, 80, 100]:
    sample_means = list()
    for i in range(times):
        sample = bootstrapResample(dataset, ratio)
        sample_mean = mean([row[0] for row in sample])
        sample_means.append(sample_mean)
    print (('Samples=%d, Estimated Mean: % 3f' % (times,mean(sample_means))))

True Mean: 4.000
Samples=20, Estimated Mean:  4.400000
Samples=40, Estimated Mean:  3.615000
Samples=60, Estimated Mean:  4.043333
Samples=80, Estimated Mean:  4.150000
Samples=100, Estimated Mean:  4.126000


In [4]:
a = [row[0] for row in dataset]

In [6]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt

# 读取csv数据
def load_csv(filename):
    dataset = []
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# 将列中的数据格式从string变换为float
def str_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
        
# 将列中的数据格式从string变换为integer
def str_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = {}
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# 将数据分成k份
def cross_validation_split(dataset, n_folds):
    dataset_split = []
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = []
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# 计算准确率
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# 通过交叉验证评估算法
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = []
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = []
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

# 根据一个属性和属性的值划分数据集
def test_split(index, value, dataset):
    left, right = [], []
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right
    
# 计算基尼系数划分数据集
def gini_index(groups, class_values):
    gini = 0.0
    for class_value in class_values:
        for group in groups:
            size = len(group)
            if size == 0:
                continue
            proportion = [row[-1] for row in group].count(class_value) / float(size)
            gini += (proportion * (1.0 - proportion))
    return gini

# 选择数据集中最好的划分点
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

# 创建一个结束节点值
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# 创建一个子节点的划分或者结束划分
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # 检查不再划分
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left+right)
        return
    # 检查最大树深度
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)
        
# 创建决策树
def build_tree(train, max_depth, min_size):
    root = get_split(dataset)
    split(root, max_depth, min_size, 1)
    return root

# 通过决策树做出预测
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']
        
def subsample(dataset, ratio):
    sample = []
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

def bagging(train, test, max_depth, min_size, sample_size, n_trees):
    trees = []
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return (predictions)

# 使用sonar数据集测试bagging算法
seed(1)
filename = 'E:/Kaggle/sonar.all-data'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
    str_to_float(dataset, i)
str_to_int(dataset, len(dataset[0])-1)
# 评估算法
n_folds = 5
max_depth = 6
min_size = 2
sample_size = 0.50
for n_trees in [1, 5, 10, 50]:
    scores = evaluate_algorithm(dataset, bagging, n_folds, max_depth, min_size, sample_size, n_trees)
    print ('Trees: %d' % n_trees)
    print ('Scores: %s' % scores)
    print ('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))


Trees: 1
Scores: [75.60975609756098, 75.60975609756098, 65.85365853658537, 63.41463414634146, 51.21951219512195]
Mean Accuracy: 66.341%
Trees: 5
Scores: [68.29268292682927, 80.48780487804879, 73.17073170731707, 56.09756097560976, 53.65853658536586]
Mean Accuracy: 66.341%
Trees: 10
Scores: [73.17073170731707, 75.60975609756098, 65.85365853658537, 58.536585365853654, 58.536585365853654]
Mean Accuracy: 66.341%
Trees: 50
Scores: [73.17073170731707, 63.41463414634146, 53.65853658536586, 65.85365853658537, 78.04878048780488]
Mean Accuracy: 66.829%
